<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Exploratory-Data-Analysis" data-toc-modified-id="Exploratory-Data-Analysis-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Exploratory Data Analysis</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Number-of-articles-in-each-category" data-toc-modified-id="Number-of-articles-in-each-category-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>Number of articles in each category</a></span></li><li><span><a href="#%-of-articles-in-each-category" data-toc-modified-id="%-of-articles-in-each-category-1.0.2"><span class="toc-item-num">1.0.2&nbsp;&nbsp;</span>% of articles in each category</a></span></li><li><span><a href="#News-length-by-category" data-toc-modified-id="News-length-by-category-1.0.3"><span class="toc-item-num">1.0.3&nbsp;&nbsp;</span>News length by category</a></span></li></ul></li></ul></li><li><span><a href="#Pre-processing-&amp;-Feature-Engineering" data-toc-modified-id="Pre-processing-&amp;-Feature-Engineering-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Pre-processing &amp; Feature Engineering</a></span><ul class="toc-item"><li><span><a href="#Coding-the-labels" data-toc-modified-id="Coding-the-labels-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Coding the labels</a></span></li><li><span><a href="#Create-the-train-and-test-sets" data-toc-modified-id="Create-the-train-and-test-sets-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Create the train and test sets</a></span></li><li><span><a href="#Create-features-using-TF-IDF" data-toc-modified-id="Create-features-using-TF-IDF-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Create features using TF-IDF</a></span></li></ul></li><li><span><a href="#Build-Models" data-toc-modified-id="Build-Models-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Build Models</a></span><ul class="toc-item"><li><span><a href="#Random-Forest" data-toc-modified-id="Random-Forest-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Random Forest</a></span></li><li><span><a href="#Model-fit-and-perforemance" data-toc-modified-id="Model-fit-and-perforemance-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Model fit and perforemance</a></span></li></ul></li></ul></div>

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
sns.set_style("whitegrid")
#import altair as alt
#alt.renderers.enable("notebook")

# Code for hiding seaborn warnings
import warnings
warnings.filterwarnings("ignore")


import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2

from sklearn.ensemble import RandomForestClassifier
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import ShuffleSplit

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')


Loading the dataset:

In [ ]:
url = '/Users/jimcody/Documents/2021Python/nlp/data/News_dataset.csv'
df = pd.read_csv(url, sep=';')

In [ ]:
df.head()

### Number of articles in each category

In [ ]:
sns.countplot(x="Category", data=df)

### News length by category

Definition of news length field. Although there are special characters in the text (``\r, \n``), it will be useful as an approximation.

In [ ]:
df['News_length'] = df['Content'].str.len()

In [ ]:
plt.figure(figsize=(12.8,6))
sns.distplot(df['News_length']).set_title('News length distribution');

In [ ]:
df['News_length'].describe()

Let's remove from the 95% percentile onwards to better appreciate the histogram:

In [ ]:
quantile_95 = df['News_length'].quantile(0.95)
df_95 = df[df['News_length'] < quantile_95]

In [ ]:
plt.figure(figsize=(12.8,6))
sns.distplot(df_95['News_length']).set_title('News length distribution');

We can get the number of news articles with more than 10,000 characters:

In [ ]:
df_more10k = df[df['News_length'] > 10000]
len(df_more10k)

Let's see one:

In [ ]:
df_more10k['Content'].iloc[0]

It's just a large news article.

Let's now plot a boxplot:

In [ ]:
plt.figure(figsize=(12.8,6))
sns.boxplot(data=df, x='Category', y='News_length', width=.5);

Now, let's remove the larger documents for better comprehension:

In [ ]:
plt.figure(figsize=(12.8,6))
sns.boxplot(data=df_95, x='Category', y='News_length');

We can see that, although the length distribution is different for every category, the difference is not too big. If we had way too different lengths between categories we would have a problem since the feature creation process may take into account counts of words. However, when creating the features with TF-IDF scoring, we will normalize the features just to avoid this.

At this point, we cannot do further Exploratory Data Analysis. We'll turn onto the **Feature Engineering** section.

We'll save the dataset:

In [ ]:
#with open('News_dataset.pickle', 'wb') as output:
#    pickle.dump(df, output)

# Pre-processing & Feature Engineering

In [ ]:
#path_df = "/home/lnc/0. Latest News Classifier/02. Exploratory Data Analysis/News_dataset.pickle"

#path_df ='/Users/jimcody/Documents/2021Python/nlp/data/News_dataset.pickle'
#with open(path_df, 'rb') as data:
#    df = pickle.load(data)

In [ ]:
df.head()

In [ ]:
# \r and \n
df['Content_Parsed_1'] = df['Content'].str.replace("\r", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("\n", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("    ", " ")

df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('"', '')  # remove double-quote
df['Content_Parsed_2'] = df['Content_Parsed_1'].str.lower()           # make lowercase

punctuation_signs = list("?:!.,;")
df['Content_Parsed_3'] = df['Content_Parsed_2']                       # remove punctuation
for punct_sign in punctuation_signs:
    df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')
    
df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("'s", "") # # Possessive pronouns

In [ ]:
# Lemmatize
wordnet_lemmatizer = WordNetLemmatizer()

# Iterate over every word in order to lemmatize
nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):
    
    # Create an empty list containing lemmatized words
    lemmatized_list = []
    
    # Save the text and its words into an object
    text = df.loc[row]['Content_Parsed_4']
    text_words = text.split(" ")

    # Iterate through every word to lemmatize
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        
    # Join the list
    lemmatized_text = " ".join(lemmatized_list)
    
    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)
    
df['Content_Parsed_5'] = lemmatized_text_list

In [ ]:
# Stop words
stop_words = list(stopwords.words('english'))

df['Content_Parsed_6'] = df['Content_Parsed_5']     # Put 5 into 6

for stop_word in stop_words:                        # Replace 6 with blank if it is a stopword

    regex_stopword = r"\b" + stop_word + r"\b"
    df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')

# This might result in some double and triple spacing between words.  
# That will be corrected when the content is tokenized.

In [ ]:
df.head(1)

In [ ]:
list_columns = ["File_Name", "Category", "Complete_Filename", "Content", "Content_Parsed_6"]
df = df[list_columns]

df = df.rename(columns={'Content_Parsed_6': 'Content_Parsed'})

In [ ]:
df.head()

## Coding the labels

In [ ]:
category_codes = {
    'business': 0,
    'entertainment': 1,
    'politics': 2,
    'sport': 3,
    'tech': 4
}

# Category mapping
df['Category_Code'] = df['Category']
df = df.replace({'Category_Code':category_codes})
df.head()

## Create the train and test sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['Content_Parsed'], 
                                                    df['Category_Code'], 
                                                    test_size=0.15, 
                                                    random_state=8)

## Create features using TF-IDF

We have various options:

* Count Vectors as features
* TF-IDF Vectors as features
* Word Embeddings as features
* Text / NLP based features
* Topic Models as features

We have to define the different parameters:

* `ngram_range`: We want to consider both unigrams and bigrams.
* `max_df`: When building the vocabulary ignore terms that have a document
    frequency strictly higher than the given threshold
* `min_df`: When building the vocabulary ignore terms that have a document
    frequency strictly lower than the given threshold.
* `max_features`: If not None, build a vocabulary that only consider the top
    max_features ordered by term frequency across the corpus.

In [ ]:
# Parameter election
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300

In [ ]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

In [ ]:
from sklearn.feature_selection import chi2
import numpy as np

for Product, category_id in sorted(category_codes.items()):
    features_chi2 = chi2(features_train, labels_train == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}' category:".format(Product))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-5:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-2:])))
    print("")


In [ ]:
bigrams

# Save the data & features

In [ ]:
# X_train
with open('/Users/jimcody/Documents/2021Python/nlp/data/Pickles/X_train.pickle', 'wb') as output:
    pickle.dump(X_train, output)
    
# X_test    
with open('/Users/jimcody/Documents/2021Python/nlp/data/Pickles/X_test.pickle', 'wb') as output:
    pickle.dump(X_test, output)
    
# y_train
with open('/Users/jimcody/Documents/2021Python/nlp/data/Pickles/y_train.pickle', 'wb') as output:
    pickle.dump(y_train, output)
    
# y_test
with open('/Users/jimcody/Documents/2021Python/nlp/data/Pickles/y_test.pickle', 'wb') as output:
    pickle.dump(y_test, output)
    
# df
with open('/Users/jimcody/Documents/2021Python/nlp/data/Pickles/df.pickle', 'wb') as output:
    pickle.dump(df, output)
    
# features_train
with open('/Users/jimcody/Documents/2021Python/nlp/data/Pickles/features_train.pickle', 'wb') as output:
    pickle.dump(features_train, output)

# labels_train
with open('/Users/jimcody/Documents/2021Python/nlp/data/Pickles/labels_train.pickle', 'wb') as output:
    pickle.dump(labels_train, output)

# features_test
with open('/Users/jimcody/Documents/2021Python/nlp/data/Pickles/features_test.pickle', 'wb') as output:
    pickle.dump(features_test, output)

# labels_test
with open('/Users/jimcody/Documents/2021Python/nlp/data/Pickles/labels_test.pickle', 'wb') as output:
    pickle.dump(labels_test, output)
    
# TF-IDF object
with open('/Users/jimcody/Documents/2021Python/nlp/data/Pickles/tfidf.pickle', 'wb') as output:
    pickle.dump(tfidf, output)